<h2>IMPORTING THE REQUIRED LIBRARIES</h2>

In [1]:
from getpass import getpass
from mysql.connector import connection

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

pd.set_option('display.max_columns', None)

<h2>CREATING A CONNECTION BETWEEN OUR JUPYTER NOTEBOOK AND MYSQL. SELECTING THE SAKILA DATABASE</h2>

In [2]:
def mysql_database_connection(user, host, database):

    password = getpass()

    mysql_connection = connection.MySQLConnection(user = user, password = password,
                                                  host = host, database = database)
                                     
    return mysql_connection

sakila_engine = mysql_database_connection('root', 'localhost', 'sakila')

<h2>EXTRACTING THE UNIQUE FILM TITLES THAT EXIST WITHIN THE DATABASE</h2>

In [3]:
def execute_query(query, engine):

    return pd.read_sql_query(query, engine)

query_film_titles = '''SELECT DISTINCT title FROM film'''

unique_film_titles = execute_query(query_film_titles, sakila_engine)

<h2>EXTRACTING THE FILMS THAT HAS BEEN RENTED IN AUGUST AND THE NUMBER OF TIMES THAT THIS EVENT HAS HAPPENED</h2>

In [4]:
def films_rented_in_august(query, engine):

    dataframe = execute_query(query, engine)
    dataframe['rental_month'] = dataframe['rental_date'].dt.month
    dataframe = dataframe[dataframe['rental_month'] == 8]
    dataframe = dataframe.groupby('title').agg({'rental_month':len}).reset_index()
    dataframe = dataframe.rename(columns = {'rental_month': 'rentals_august'})

    return dataframe

query_film_rentals = '''SELECT f.title, r.rental_date
                        FROM film f JOIN inventory i USING(film_id)
                        JOIN rental r ON i.inventory_id = r.inventory_id'''

rentals_in_august_per_film = films_rented_in_august(query_film_rentals, sakila_engine)

<h2>JOINING THE UNIQUE FILM TITLES WITH THE VALUE OF TIMES THAT EVERY TITLE WAS RENTED IN AUGUST - DEFINING THE TARGET COLUMN</h2>

In [5]:
def map_films_rentals_august(dataframe_1, dataframe_2):

    dataframe_1_dict = dataframe_1.set_index('title')['rentals_august'].to_dict()
    dataframe_2['rentals_in_august'] = dataframe_2['title'].map(dataframe_1_dict)
    dataframe_2['rentals_in_august'] = dataframe_2['rentals_in_august'].fillna(0)
    
    def rental_august(x):

        if x > 0:
            return 1
        
        else:
            return 0

    dataframe_2['rented_in_august'] = dataframe_2['rentals_in_august'].apply(rental_august)
    dataframe_2 = dataframe_2.drop('rentals_in_august', axis = 1)

    return dataframe_2

rented_or_not_august = map_films_rentals_august(rentals_in_august_per_film, unique_film_titles)

<h2>EXTRACTING THE FILM DATA AND MERGING THE DATA WITH THE RENTED IN AUGUST CONDITION</h2>

In [6]:
query_film_data = '''SELECT title, rental_duration, rental_rate, length, 
                     replacement_cost, rating, special_features
                     FROM film'''

films_data = execute_query(query_film_data, sakila_engine)
film_data_target = pd.merge(films_data, rented_or_not_august)

<h2>DROPPING THE TITLE COLUMN ONCE WE ALREADY HAVE THE DATA</h2>

In [7]:
film_data_target = film_data_target.drop('title', axis = 1)

<h2>FORMATTING THE SPECIAL FEATURES COLUMN BEFORE USING THE FEATURE</h2>

In [8]:
def order_special_features(x):

    return sorted(x, key = str.lower)

film_data_target['special_features'] = film_data_target['special_features'].apply(order_special_features)

<h2>MANUALLY ONE HOT ENCODING THE SPECIAL FEATURES COLUMN</h2>

In [9]:
def one_hot_special_features(dataframe, column):

    dataframe['behind_the_scenes'] = 0
    dataframe['commentaries'] = 0
    dataframe['deleted_scenes'] = 0
    dataframe['trailers'] = 0

    def behind_the_scenes(row):

        if 'Behind the Scenes' in row[column]:
            return 1

        return 0

    def commentaries(row):

        if 'Commentaries' in row[column]:
            return 1

        return 0

    def deleted_scenes(row):

        if 'Deleted Scenes' in row[column]:
            return 1

        return 0

    def trailers(row):

        if 'Trailers' in row[column]:
            return 1

        return 0

    dataframe['behind_the_scenes'] = film_data_target.apply(behind_the_scenes, axis = 1)
    dataframe['commentaries'] = film_data_target.apply(commentaries, axis = 1) 
    dataframe['deleted_scenes'] = film_data_target.apply(deleted_scenes, axis = 1)
    dataframe['trailers'] = film_data_target.apply(trailers, axis = 1)

    dataframe = dataframe.drop(column, axis = 1)

    return dataframe

film_data_target = one_hot_special_features(film_data_target, 'special_features')

<h2>CONVERTING THE RENTAL DURATION AND THE RENTAL RATE COLUMNS IN OBJECTS</h2>

In [10]:
def numerical_to_categorical(dataframe, columns):

    for column in columns:
        dataframe[column] = dataframe[column].astype('object')

    return dataframe

film_data_target = numerical_to_categorical(film_data_target, ['rental_duration', 'rental_rate'])

<h2>PERFORMING THE X-Y SPLIT AND THE TRAIN TEST SPLIT</h2>

In [11]:
def x_y_train_test(dataframe, target):

    X = dataframe.drop(target, axis = 1)
    y = dataframe[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = x_y_train_test(film_data_target, 'rented_in_august')

<h2>SCALING THE NUMERICAL FEATURES</h2>

In [12]:
def scaling(training_x, testing_x, columns):

    training_x_numerical = training_x[columns]
    testing_x_numerical = testing_x[columns]

    scaler = StandardScaler().fit(training_x_numerical)

    training_x_numerical_array = scaler.transform(training_x_numerical)
    training_x_numerical_scaled = pd.DataFrame(data = training_x_numerical_array, columns = training_x_numerical.columns, index = training_x.index)

    testing_x_numerical_array = scaler.transform(testing_x_numerical)
    testing_x_numerical_scaled = pd.DataFrame(data = testing_x_numerical_array, columns = testing_x_numerical.columns, index = testing_x.index)

    numerical_columns = training_x_numerical.columns.to_list()
    training_x[numerical_columns] = training_x_numerical_scaled
    testing_x[numerical_columns] = testing_x_numerical_scaled

    return training_x, testing_x

X_train_scaled, X_test_scaled = scaling(X_train, X_test, ['length', 'replacement_cost'])

<h2>ONE HOT ENCODING THE CATEGORICAL FEATURES</h2>

In [13]:
def one_hot_encoding(training_x, testing_x, columns):

    training_x_categorical = training_x[columns]
    testing_x_categorical = testing_x[columns]

    training_x = training_x.drop(columns, axis = 1)
    testing_x = testing_x.drop(columns, axis = 1)

    encoder = OneHotEncoder().fit(training_x_categorical)

    total_columns = list()
    columns_to_drop = list()

    for columns in encoder.categories_:
        columns_to_drop.append(columns[0])
        for column in columns:
            total_columns.append(column)

    training_x_categorical_array = encoder.transform(training_x_categorical).toarray()
    training_x_categorical_encoded = pd.DataFrame(data = training_x_categorical_array, columns = total_columns, index = training_x.index)
    training_x_categorical_encoded = training_x_categorical_encoded.drop(columns_to_drop, axis = 1)

    testing_x_categorical_array = encoder.transform(testing_x_categorical).toarray()
    testing_x_categorical_encoded = pd.DataFrame(data = testing_x_categorical_array, columns = total_columns, index = testing_x.index)
    testing_x_categorical_encoded = testing_x_categorical_encoded.drop(columns_to_drop, axis = 1)

    training_x = pd.concat([training_x, training_x_categorical_encoded], axis = 1)
    testing_x = pd.concat([testing_x, testing_x_categorical_encoded], axis = 1)

    return training_x, testing_x

X_train_scaled_encoded, X_test_scaled_encoded = one_hot_encoding(X_train_scaled, X_test_scaled, ['rental_duration', 'rental_rate', 'rating'])

<h2>OVER SAMPLING THE MINORITY CLASS OF THE TARGET FEATURE</h2>

In [14]:
def over_sampling(training_x, training_y):

    smote = SMOTE(random_state = 100, k_neighbors = 3)
    X_train_scaled_SMOTE, y_train_SMOTE = smote.fit_resample(training_x, training_y)

    return X_train_scaled_SMOTE, y_train_SMOTE    

X_train_scaled_encoded_SMOTE, y_train_SMOTE = over_sampling(X_train_scaled_encoded, y_train)

<h2>CREATING AND FITTING A LOGISTIC REGRESSION METHOD</h2>

In [15]:
def create_fit_logistic_regression(training_x, training_y):

    model = LogisticRegression()
    model.fit(training_x, training_y)
    
    return model

logistic_regression = create_fit_logistic_regression(X_train_scaled_encoded_SMOTE, y_train_SMOTE)

<h2>EXTRACTING THE METRICS OF THE MODEL</h2>

In [16]:
def model_metrics(model, testing_x, testing_y):

    predictions = model.predict(testing_x)

    accuracy = accuracy_score(testing_y, predictions)
    precision = precision_score(testing_y, predictions)
    recall = recall_score(testing_y, predictions)
    f1 = f1_score(testing_y, predictions)

    return accuracy, precision, recall, f1

model_accuracy, model_precision, model_recall, model_f1 = model_metrics(logistic_regression, X_test_scaled_encoded, y_test)

<h2>EVALUATING THE METRICS OF THE MODEL</h2>

In [17]:
print('\nLOGISTIC REGRESSION METRICS')
print('---------------------------\n')

print(f'- Accuracy: {round(model_accuracy, 2)}')
print(f'- Precision: {round(model_precision, 2)}')
print(f'- Recall: {round(model_recall, 2)}')
print(f'- F1: {round(model_f1, 2)}')


LOGISTIC REGRESSION METRICS
---------------------------

- Accuracy: 0.68
- Precision: 0.96
- Recall: 0.7
- F1: 0.81
